In [1]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# input tickers of your choice up to 100 at a time per rules of yahoo finance API
tickers = ['CWB','BSV','DES','BIV','PFLT','AGG','HYG','BND',
           'SPHD','LQD','SPLV','PFF','GAIN','ELD','BLV','MDIV','EMB','SJR',
           'AGNC','HRZN','IYLD','JNK','BKLN','GLAD','MAIN','GOOD',
           'SCM','GWRS','FLOT','DX','LAND','SBR','SUNS','O','PSEC','PBA',
           'ERF','LTC','ORC','APLE','SJT','SDIV','ARR','CRT','WSR','STAG']

infos = []

for i in tickers:
    infos.append(yf.Ticker(i).info)
    
# get dividends, stock price, and company name     
fundamentals =  ['longName','regularMarketPrice','yield','dividendYield']

df = pd.DataFrame(infos)

df = df.set_index('symbol')

stock_list = df[df.columns[df.columns.isin(fundamentals)]]

# get daily price information from yahoo finance and enter time period of choice
stock_prices = yf.download(tickers, '2017-02-07','2022-02-07')['Adj Close']

# calculate daily price returns 
rets = stock_prices.pct_change()

# calculate cumulative returns for time period analyzed
cumulative_return = (1 + rets ).cumprod() - 1
total_ret = cumulative_return.iloc[[-1]].T
total_ret = total_ret.rename(columns={total_ret.columns[0]: "Total 5 Year Return"})

# calculate largest 1 day price drop for time period analyzed
rets = stock_prices.pct_change()
largest_1_day_drop = rets.describe().iloc[[-5]].T
largest_1_day_drop = largest_1_day_drop.rename(columns={largest_1_day_drop.columns[0]: "Largest 1 day Drop"})

# calculate largest 5 day price drop for time period analyzed

#five_day_rets = five_day_stock_prices.pct_change()
five_day_rets = stock_prices.pct_change(5)
largest_5_day_drop = five_day_rets.describe().iloc[[-5]].T
largest_5_day_drop = largest_5_day_drop .rename(columns={largest_5_day_drop .columns[0]: "Largest 5 day Drop"})

# calculate annualized Risk, Return, and Sharpe Ratio

risk_free_return = 0
risk_free_risk = 0
rf = [risk_free_return, risk_free_risk]

# calculate annualized Risk and Return
def ann_risk_return(returns_df):
    summary = returns_df.agg(["mean", "std"]).T
    summary.columns = ["Return", "Risk"]
    summary.Return = summary.Return*252
    summary.Risk = summary.Risk * np.sqrt(252)
    return summary

summary = ann_risk_return(rets)

# calculate sharpe ratio or risk adjusted return
# assuming riskfree rate of 0 for analysis purposes
summary["Sharpe Ratio"] = (summary["Return"].sub(rf[0]))/summary["Risk"]

#download list into dataframe
new_list1 = stock_list.join(total_ret)
new_list2 = new_list1.join(largest_1_day_drop)
new_list3 = new_list2.join(largest_5_day_drop)
final_list = new_list3.join(summary)

# clean final output
final_list = final_list.rename(columns={final_list.columns[0]: "Ticker Name"})
final_list = final_list.rename(columns={final_list.columns[3]: "Current Stock Price"})
final_list = final_list.rename(columns={final_list.columns[8]: "Annualized Volatility"})
final_list['Dividend Yield'] = np.where(final_list['yield'].isna(), final_list['dividendYield'], final_list['yield'])
final_list = final_list[["Ticker Name","Dividend Yield","Current Stock Price","Total 5 Year Return","Largest 1 day Drop","Largest 5 day Drop","Annualized Volatility","Sharpe Ratio"]]
final_list

[*********************100%***********************]  46 of 46 completed


,Ticker Name,Dividend Yield,Current Stock Price,Total 5 Year Return,Largest 1 day Drop,Largest 5 day Drop,Annualized Volatility,Sharpe Ratio
symbol,,,,,,,,
CWB,SPDR Bloomberg Barclays Convertible Securities...,0.0214,77.4400,0.926166,-0.091328,-0.187500,0.162078,0.892136
BSV,Vanguard Short-Term Bond Index Fund ETF Shares,0.0137,79.2500,0.094265,-0.016502,-0.035553,0.019622,0.929484
DES,WisdomTree U.S. SmallCap Dividend Fund,0.0206,32.0700,0.346547,-0.127157,-0.255570,0.243579,0.367550
BIV,Vanguard Intermediate-Term Bond Index Fund ETF...,0.0198,84.0400,0.174552,-0.023954,-0.050295,0.044309,0.749563
PFLT,PennantPark Floating Rate Capital Ltd.,0.0866,13.2900,0.516625,-0.295113,-0.605617,0.353073,0.419007
AGG,iShares Core U.S. Aggregate Bond ETF,0.0195,109.4100,0.148573,-0.040010,-0.071043,0.046762,0.616943
HYG,iShares iBoxx $ High Yield Corporate Bond ETF,0.0444,83.1200,0.225886,-0.054966,-0.128669,0.087246,0.511229
BND,Vanguard Total Bond Market Index Fund ETF Shares,0.0195,81.2100,0.155648,-0.054385,-0.080472,0.050397,0.600349
SPHD,Invesco S&P 500 High Dividend Low Volatility ETF,0.0371,45.1500,0.386321,-0.120303,-0.226178,0.205924,0.421604


In [2]:
final_list.to_csv('top_dividend_stocks.csv')

In [3]:
# Get Dividend Pay Periods

import pandas as pd
import yfinance as yf
import datetime as dt

##tickers = pd.read_csv('Dow_Stocks.csv')

tickers = ['CWB','BSV','DES','BIV','PFLT','AGG','HYG','BND',
           'SPHD','LQD','SPLV','PFF','GAIN','ELD','BLV','MDIV','EMB','SJR',
           'AGNC','HRZN','IYLD','JNK','BKLN','GLAD','MAIN','GOOD',
           'SCM','GWRS','FLOT','DX','LAND','SBR','SUNS','O','PSEC','PBA',
           'ERF','LTC','ORC','APLE','SJT','SDIV','ARR','CRT','WSR','STAG']

divs = []

for i in tickers:
    inst = yf.Ticker(i)
    inst.history(period='1y')
    divs.append(inst.dividends)
    
df = pd.DataFrame(divs, index=tickers)

df.columns = df.columns.month

df = df.groupby(df.columns, axis=1).sum()

df.columns = [dt.date(1900, i,1).strftime('%b') for i in df.columns]

df.to_excel('Dividends.xlsx')

In [4]:
df

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
CWB,0.000,0.044000,0.122000,0.094000,0.045000,0.125000,0.092000,0.043000,0.134000,0.090,0.045000,0.797
BSV,0.000,0.074000,0.084000,0.089000,0.084000,0.083000,0.078000,0.072000,0.076000,0.073,0.073000,0.366
DES,0.010,0.010000,0.001000,0.020000,0.045000,0.110000,0.050000,0.080000,0.090000,0.050,0.095000,0.199
BIV,0.000,0.139000,0.138000,0.461000,0.143000,0.148000,0.141000,0.145000,0.143000,0.138,0.141000,1.248
PFLT,0.095,0.095000,0.095000,0.095000,0.095000,0.095000,0.095000,0.095000,0.095000,0.095,0.095000,0.095
AGG,0.000,0.161000,0.190000,0.172000,0.173000,0.167000,0.156000,0.167000,0.167000,0.180,0.180000,0.279
HYG,0.000,0.305000,0.307000,0.294000,0.294000,0.291000,0.292000,0.283000,0.275000,0.289,0.277000,0.578
BND,0.000,0.137000,0.129000,0.192000,0.131000,0.134000,0.132000,0.136000,0.135000,0.130,0.134000,0.406
SPHD,0.139,0.145000,0.141000,0.126000,0.114000,0.121000,0.114000,0.127000,0.000000,0.130,0.133000,0.137
LQD,0.000,0.242000,0.269000,0.254000,0.255000,0.257000,0.253000,0.243000,0.234000,0.255,0.262000,0.495
